In [4]:
import numpy as np

In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Загрузка данных
data = pd.read_csv('data/root_cause_analysis.csv')
X = data.iloc[:, 1:-1].values
y = data.iloc[:, -1].values
le = LabelEncoder()
y = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание модели
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(7, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

# Обучение модели
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
for epoch in range(1000):
    optimizer.zero_grad()
    output = net(torch.tensor(X_train, dtype=torch.float))
    loss = criterion(output, torch.tensor(y_train, dtype=torch.long))
    loss.backward()
    optimizer.step()

# Оценка модели
y_pred = torch.argmax(net(torch.tensor(X_test, dtype=torch.float)), dim=1).numpy()
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred, average='macro'))
print('Recall:', recall_score(y_test, y_pred, average='macro'))
print('F1-score:', f1_score(y_test, y_pred, average='macro'))


Accuracy: 0.85
Precision: 0.8525715663646699
Recall: 0.8495370370370371
F1-score: 0.8500740452652679


In [21]:
import numpy as np

new_data = np.array([
    [0, 1, 0, 1, 1, 0, 1],  # MEMORY
    [1, 0, 1, 0, 0, 1, 0],  # NETWORK_DELAY
    [1, 1, 0, 0, 1, 0, 0],  # DATABASE_ISSUE
    [0, 0, 1, 0, 1, 1, 0],  # NETWORK_DELAY
    [0, 1, 0, 1, 0, 1, 0]   # MEMORY
])

# Применение модели к новым данным
y_pred = torch.argmax(net(torch.tensor(new_data, dtype=torch.float)), dim=1).numpy()
y_pred = le.inverse_transform(y_pred)
print(y_pred)

# Применение модели к новым данным и вывод вероятностей предсказаний
y_pred_prob = torch.softmax(net(torch.tensor(new_data, dtype=torch.float)), dim=1).detach().numpy()
print(y_pred_prob)



['MEMORY' 'DATABASE_ISSUE' 'DATABASE_ISSUE' 'NETWORK_DELAY' 'MEMORY']
[[2.0287104e-04 9.9806029e-01 1.7368564e-03]
 [7.8804922e-01 2.1144931e-06 2.1194862e-01]
 [4.9989614e-01 4.9952051e-01 5.8345060e-04]
 [1.7901069e-03 4.4402668e-06 9.9820542e-01]
 [3.3635265e-04 9.9249929e-01 7.1643875e-03]]
